In [1]:
from google.colab import files
uploaded = files.upload()


Saving TrainAndValid.csv to TrainAndValid.csv
Saving Data Dictionary.xlsx to Data Dictionary.xlsx
Saving Train.csv to Train.csv
Saving Machine_Appendix.csv to Machine_Appendix.csv
Saving median_benchmark.csv to median_benchmark.csv
Saving random_forest_benchmark_test.csv to random_forest_benchmark_test.csv
Saving Test.csv to Test.csv
Saving Valid.csv to Valid.csv
Saving ValidSolution.csv to ValidSolution.csv


Bulldozer Price Prediction

Data Exploration

In [14]:
import pandas as pd

# Load datasets
train_df = pd.read_csv("Train.csv", low_memory=False)
valid_df = pd.read_csv("Valid.csv", low_memory=False)
train_valid_df = pd.read_csv("TrainAndValid.csv", low_memory=False)
test_df = pd.read_csv("Test.csv", low_memory=False)
valid_solution = pd.read_csv("ValidSolution.csv")

# Benchmarks
rf_benchmark = pd.read_csv("random_forest_benchmark_test.csv")
median_benchmark = pd.read_csv("median_benchmark.csv")

# Preview training data
train_df.head()


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data Cleaning and Preprocessing

In [15]:
def preprocess(df):
    df = df.copy()
    object_cols = df.select_dtypes(include=['object']).columns
    df = df.drop(columns=object_cols)
    return df

# Split features and target
X_train = preprocess(train_df.drop('SalePrice', axis=1))
y_train = train_df['SalePrice']

X_valid = preprocess(valid_df)
y_valid = valid_solution['SalePrice']
X_test = preprocess(test_df)


Model Building

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

# Define RMSLE function
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, np.maximum(0, y_pred)))

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Validate
val_preds = model.predict(X_valid)
val_score = rmsle(y_valid, val_preds)
print(f"✅ Validation RMSLE: {val_score:.4f}")


✅ Validation RMSLE: 0.3291


Train on TrainAndValid data for better predictions

In [17]:
#Train on full data for better predictions
use_full_data = False  # Set to True before final run

if use_full_data:
    full_df = pd.read_csv("TrainAndValid.csv", low_memory=False)
    X_full = preprocess(full_df.drop('SalePrice', axis=1))
    y_full = full_df['SalePrice']
    model.fit(X_full, y_full)
    print("✅ Trained on full TrainAndValid.csv for final test predictions.")
else:
    model.fit(X_train, y_train)


 Predictions on Test Data

In [21]:
# Predict on test
test_preds = model.predict(X_test)

# Export predictions
submission = pd.DataFrame({
    'SalesID': test_df['SalesID'],
    'SalePrice': test_preds
})
submission.to_csv("test_predictions.csv", index=False)
print("test_predictions.csv created successfully.")


test_predictions.csv created successfully.


In [20]:
# Compare with Random Forest Benchmark (on TEST set)
merged_rf = pd.merge(rf_benchmark, submission, on="SalesID", suffixes=('_rf', '_your'))
print(f"Merged with RF Benchmark: {len(merged_rf)} rows")

rf_rmsle = rmsle(merged_rf['SalePrice_rf'], merged_rf['SalePrice_your'])
print(f" RMSLE vs RF Benchmark (Test): {rf_rmsle:.4f}")

# Compare with Median Benchmark (on VALID set)
valid_preds_df = pd.DataFrame({
    'SalesID': valid_df['SalesID'],
    'SalePrice': val_preds
})
merged_median = pd.merge(median_benchmark, valid_preds_df, on="SalesID", suffixes=('_median', '_your'))
print(f"Merged with Median Benchmark: {len(merged_median)} rows")

if not merged_median.empty:
    median_rmsle = rmsle(merged_median['SalePrice_median'], merged_median['SalePrice_your'])
    print(f"RMSLE vs Median Benchmark (Validation): {median_rmsle:.4f}")
else:
    print("Could not merge with median benchmark (check SalesID alignment).")


Merged with RF Benchmark: 12457 rows
 RMSLE vs RF Benchmark (Test): 0.2805
Merged with Median Benchmark: 11573 rows
RMSLE vs Median Benchmark (Validation): 0.6660


In [22]:
from google.colab import files
files.download('test_predictions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>